# Pixel-wise distance

Everything in grayscale

In [7]:
import numpy as np

In [8]:
def distance(p1, p2, coeff = 2):
    diff = ((p1 - p2) ** 2) ** 0.7
    return np.minimum(diff * coeff, np.ones(p1.shape) * 255)

# Denoising

In [9]:
from scipy import signal
def little_blur(img):
    kernel = np.ones((3, 3)) / 9.
    return signal.convolve2d(img, kernel)

In [10]:
from scipy import ndimage
def median(img, size=5):
    return ndimage.median_filter(img, size)

# Hull correction

In [11]:
from queue import Queue
def find_component(matrix, start, used):
    h, w = matrix.shape
    result = []
    v = matrix[start]

    queue = Queue()
    queue.put(start)
    used[start] = True

    while not queue.empty():
        x, y = queue.get()
        result.append((x, y))
        for dx in range(max(x-1, 0), min(x+2, h)):
            for dy in range(max(y-1, 0), min(y+2, w)):
                if matrix[dx, dy] == v and not used[dx, dy]:
                    queue.put((dx, dy))
                    used[dx, dy] = True
    return result

In [12]:
from scipy.spatial import ConvexHull
from scipy.spatial import QhullError

In [13]:
from scipy.spatial import Delaunay
def in_hull(points, hull_points):
    hull = Delaunay(hull_points)
    return hull.find_simplex(points)>=0

In [14]:
def fill_hulls_entire(mask, points, mask_opacity=80):
    points = np.asarray(points)
    mn = np.min(points, axis=0)
    mx = np.max(points, axis=0)
    rect = np.asarray([(x, y)
              for x in range(mn[0], mx[0])
              for y in range(mn[1], mx[1])
              ])

    if len(points) > 2:
        try:
            verts = ConvexHull(points).vertices
            hull = points[verts]
            is_in_hull = np.asarray(in_hull(rect, hull))
            in_hull_points = rect[np.where(is_in_hull)]
            for x, y in in_hull_points:
                mask[x, y] = mask_opacity
        except QhullError:
            print("Hull is empty. OK")

In [15]:
def hull_area(points):
    try:
        return ConvexHull(points).volume
    except QhullError:
        return 0

In [16]:
def hull_edges(points):
    try:
        return len(ConvexHull(points).vertices) - 1
    except QhullError:
        return 0

In [17]:
def filter_drops(drops, min_ratio=0.3, filling=0.5, edges_per_pixel=0.01):
    filtered = []
    for drop in drops:
        mn = np.min(drop, axis=0)
        mx = np.max(drop, axis=0)
        area = hull_area(drop)
        [h, w] = mx - mn
        box_area = w * h
        if area <= box_area * filling:
            continue
        ratio = min(h,w) / max(h, w)
        if ratio < min_ratio:
            continue
        epp = hull_edges(drop) / len(drop)
        if epp < edges_per_pixel:
            continue
        filtered.append(drop)
    return filtered

In [18]:
def fill_by_hull(mask, mask_opacity=80):
    used = np.zeros(mask.shape, dtype=np.bool_)
    drops = []
    for (i, j), m in np.ndenumerate(mask):
        if m == mask_opacity and not used[i, j]:
            drops.append(find_component(mask, (i, j), used))
    mask.fill(0)
    for drop in drops:
        fill_hulls_entire(mask, drop, mask_opacity)

In [19]:
def masked_image(background, mask):
    foreground = np.zeros((background.height, background.width, 4))
    foreground[:, :, 0].fill(255)
    foreground[:, :, 3] = mask
    foreground = Image.fromarray(foreground.astype('uint8'), "RGBA")

    background.paste(Image.new('RGB', background.size, (255, 0, 0)), mask=foreground)
    return background

# Estimation

In [ ]:
from PIL import Image

In [ ]:
clear_path = '../data/test/000/000_0.png'
dirty_path = '../data/test/000/005_0.png'
mask_path  = '../data/test/mask/000/005_0.png'
mask_opacity = 80

# load images
clear = Image.open(clear_path)
dirty = Image.open(dirty_path)
# prepare them
clear = np.asarray(clear.convert('L'))
dirty = np.asarray(dirty.convert('L'))
# Little blur to denoise
clear = little_blur(clear)
dirty = little_blur(dirty)

In [ ]:
diff = distance(clear, dirty)

In [ ]:
# Post filtration
diff = median(diff)
diff = diff[1:-1, 1:-1]

In [ ]:
# Select partition
mask = np.where(diff > 10, mask_opacity, 0)

In [ ]:
fill_by_hull(mask)

# Visualization

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(diff, cmap="gray")
plt.axis('off')

In [ ]:
plt.imshow(mask, cmap="gray")
plt.axis('off')

In [ ]:
masked = masked_image(Image.open(dirty_path), mask)

In [ ]:
from pathlib import Path
Path(mask_path).parent.mkdir(parents=True, exist_ok=True)

In [ ]:
masked.save(mask_path)

In [ ]:
plt.imshow(masked, cmap="gray")
plt.axis('off')

# Applying to all images

In [27]:
from PIL import Image
from pathlib import Path

In [34]:
def mask_drops(clear_path, dirty_path, mask_opacity=80):
    # load images
    clear = Image.open(clear_path)
    dirty = Image.open(dirty_path)
    # prepare them
    clear = np.asarray(clear.convert('L'))
    dirty = np.asarray(dirty.convert('L'))
    # Little blur to denoise
    clear = little_blur(clear)
    dirty = little_blur(dirty)
    diff = distance(clear, dirty)
    # Post filtration
    diff = median(diff)
    diff = diff[1:-1, 1:-1]
    # Select partition
    mask = np.where(diff > 10, mask_opacity, 0)
    fill_by_hull(mask, mask_opacity)
    diff_image = Image.fromarray(diff).convert('L')
    mask_image = Image.fromarray(mask).convert('L')
    return diff_image, mask_image

In [35]:
def load_stereo(dataset_dir = '../data/stereo/test'):
    mask_dir = dataset_dir + '/mask'
    pair_paths = []

    for path in Path(dataset_dir).rglob('*.png'):
        if not path.is_relative_to(mask_dir):
            # check if not clear
            if path.name[:3] != '000':
                clear_name = '000' + path.name[3:]
                clear_path = Path(str(path.parent) + '/' + clear_name)
                pair_paths.append([clear_path, path])
    return pair_paths

In [36]:
def load_derain(dataset_dir = '../data/derain/ALIGNED_PAIRS'):
    clean_dir = dataset_dir + '/CLEAN'
    dirty_dir = dataset_dir + '/CG_DROPLETS'
    pair_paths = []

    for path in Path(clean_dir).rglob('*.png'):
        path = path.relative_to(clean_dir)
        clean_path = Path(clean_dir + '/' + str(path))
        dirty_path = Path(dirty_dir + '/' + str(path))
        pair_paths.append([clean_path, dirty_path])
    return pair_paths

In [24]:
# apply for stereo dataset
dataset_dir = '../data/stereo/test'
pair_paths = load_stereo(dataset_dir)

In [37]:
# apply for derain dataset
dataset_dir = '../data/derain/ALIGNED_PAIRS'
pair_paths = load_derain(dataset_dir)

In [38]:
mask_dir = dataset_dir + '/mask'
diff_dir = dataset_dir + '/diff'
masked_dir = dataset_dir + '/masked'
Path(mask_dir).mkdir(parents=True, exist_ok=True)
Path(diff_dir).mkdir(parents=True, exist_ok=True)
Path(masked_dir).mkdir(parents=True, exist_ok=True)

In [39]:
for clear_path, path in pair_paths:
    print("Processing:", clear_path, path)
    mask_path = path.relative_to(dataset_dir)
    mask_name = '_'.join(mask_path.parts)
    diff, mask = mask_drops(clear_path, path)
    diff.save(diff_dir + '/' + mask_name)
    print("Saved", diff_dir + '/' + mask_name)
    #mask.save(mask_dir + '/' + mask_name)
    #print("Saved", mask_dir + '/' + mask_name)
    masked = masked_image(Image.open(path), mask)
    masked.save(masked_dir + '/' + mask_name)
    print("Saved", masked_dir + '/' + mask_name)

Processing: ..\data\derain\ALIGNED_PAIRS\CLEAN\left_1535201264389466.png ..\data\derain\ALIGNED_PAIRS\CG_DROPLETS\left_1535201264389466.png
Saved ../data/derain/ALIGNED_PAIRS/diff/CG_DROPLETS_left_1535201264389466.png
Saved ../data/derain/ALIGNED_PAIRS/masked/CG_DROPLETS_left_1535201264389466.png
Processing: ..\data\derain\ALIGNED_PAIRS\CLEAN\left_1535201264977680.png ..\data\derain\ALIGNED_PAIRS\CG_DROPLETS\left_1535201264977680.png
Saved ../data/derain/ALIGNED_PAIRS/diff/CG_DROPLETS_left_1535201264977680.png
Saved ../data/derain/ALIGNED_PAIRS/masked/CG_DROPLETS_left_1535201264977680.png
Processing: ..\data\derain\ALIGNED_PAIRS\CLEAN\left_1535201265565957.png ..\data\derain\ALIGNED_PAIRS\CG_DROPLETS\left_1535201265565957.png
Saved ../data/derain/ALIGNED_PAIRS/diff/CG_DROPLETS_left_1535201265565957.png
Saved ../data/derain/ALIGNED_PAIRS/masked/CG_DROPLETS_left_1535201265565957.png
Processing: ..\data\derain\ALIGNED_PAIRS\CLEAN\left_1535201266154233.png ..\data\derain\ALIGNED_PAIRS\CG_D

KeyboardInterrupt: 